In [ ]:
import pandas as pd
import os
import datetime
from datetime import timedelta, time

In [ ]:
# 選擇檔案目錄
dir_path = "Data_clears/"

# 取得目錄底下檔案
filename = os.listdir(dir_path)

# 建立儲存目錄
folderPath = 'flight_data_final'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 載入airport_info csv，並選取所需欄位
airport = pd.read_csv("airport_info_0325.csv")
airport["WBAN"] = airport["WBAN"].astype(str)
# 補滿5位數
airport["WBAN"] = airport["WBAN"].str.zfill(5)
airport = airport[["state_code", "WBAN", "timezone", "airport_code", "hub_type"]]

In [ ]:
for j in filename:
    
    df = pd.read_csv(f'{dir_path}{j}')
    df = df.merge(airport, left_on="depa_airport", right_on="airport_code", how = 'left')
    df1 = df.merge(airport, left_on = "dest_airport", right_on = "airport_code", how = 'left', suffixes=("_depa", "_dest"))

    for i in range(len(df1)):
        con_date = datetime.datetime(int(df1["dest_year"][i]),
                                     int(df1["dest_month"][i]),
                                     int(df1["dest_date"][i]),
                                     int(df1["dest_hr"][i]),
                                     int(df1["dest_min"][i]))
        # 先減掉出發地時區 + 抵達地時區。
        con_date += timedelta(hours = -(df1["timezone_depa"][i].astype('float')) + df1["timezone_dest"][i].astype('float'))
        df1["dest_year"][i] = con_date.year
        df1["dest_month"][i] = con_date.month
        df1["dest_date"][i] = con_date.day
        df1["dest_hr"][i] = con_date.hour
        df1["dest_min"][i] = con_date.minute

    df1.to_csv(f'{folderPath}/{j}', index = None)